# 1. Introduction

## 1.1 Members
 - Ardit Nela
 - Bisera Nikoloska
 - Lovro Oražem
 - Grigor Ristov

## 1.2 Goals
The past year we have been faced with a very difficult period with the first pandemic many of us have seen. With that we have a lot of problems, mainly the amount of hospitilizaed people. With our project we hope to show the amount of people affected because of this horrible pandemic, depending on the gender, age, place of residence and many other different parameters.

## 1.3 Technoligies used
Regarding the technologies we used. We are using Python, pcaxis files for the source, for libraries numpy, pandas, matplotlib for mapping out graphs and making the data we recieve usable, and for reading the pcaxis file we used an external library pyaxis.

## 1.4 Data processing
With regards to the data, we got it from the Slovenian Ministry of Public Administration, from podatki.gov with the name "Število hospitalizacij zaradi bolezni, po spolu in statističnih regijah, Slovenija, letno". The data we got was in a pcaxis file format we later reprocessed it using the pyaxis library and made it usable.

# 2. Data analysis

In [ ]:
from pyaxis import pyaxis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import json
import plotly.express as px

%matplotlib inline
plt.set_loglevel('WARNING')

## 2.1 First dataset
We begin by looking at the "Število hospitalizacij zaradi bolezni, po glavnih diagnozah (MKB-10, poglavja) in spolu, Slovenija, letno" dataset. We load it in and see what it contains.

In [ ]:
URL = "https://podatki.nijz.si/DB/NIJZ%20podatkovni%20portal/4%20Zdravstveno%20varstvo/06%20Bolni%C5%A1ni%C4%8Dne%20obravnave/1%20Hospitalizacije%20zaradi%20bolezni/BO04.px"
px = pyaxis.parse(URL, encoding='ANSI')
data = px['DATA']
data["DATA"] = data["DATA"].astype(int)

print(data.head(20))

Initially we decided to look for correlations in the data. For this we used the built-in pearson function.

In [ ]:
diagnosesArr = data["Poglavje glavne diagnoze"].astype("category").cat.codes
diagnosesArr = diagnosesArr.to_numpy()
hospitalizations = data["DATA"].to_numpy()
corr = np.corrcoef(diagnosesArr, hospitalizations)
print("Correlation between diagnoses and hospitalizations", corr[0][1])

genderArr = data["Spol"].astype("category").cat.codes
genderArr = genderArr.to_numpy()
corr = np.corrcoef(genderArr, hospitalizations)
print("Correlation between gender and hospitalizations", corr[0][1])

Then, we decided to find the 5 diseases which result in the most hospitalizations in total.

In [ ]:
diagnoses = dict()
for index, row in data.iterrows():
    if row["Poglavje glavne diagnoze"] not in diagnoses:
        diagnoses[row["Poglavje glavne diagnoze"]] = 0
    diagnoses[row["Poglavje glavne diagnoze"]] += int(row["DATA"])


diagnoses = dict(sorted(diagnoses.items(), key=lambda item: item[1]))
dKeys = list(diagnoses.keys())[-6:-1]
dVals = list(diagnoses.values())[-6:-1]
diagnoses = dict(zip(dKeys, dVals))
print(diagnoses)

We define a function that will plot the number of hospitalizations by gender and diagnosis throughout the years.

In [ ]:
def graphOverTime(diagnosis):
    tempData1 = data[data["Spol"] == "Moški"];
    tempData2 = data[data["Spol"] == "Ženski"];
    tempData1 = tempData1[tempData1["Poglavje glavne diagnoze"] == diagnosis];
    tempData2 = tempData2[tempData2["Poglavje glavne diagnoze"] == diagnosis];
    plt.figure(figsize=(10, 5));
    plt.xticks(rotation="vertical");
    plt.title(diagnosis);
    plt.plot(tempData1["Leto"], tempData1["DATA"], label="Moški");
    plt.plot(tempData2["Leto"], tempData2["DATA"], label="Ženski");
    plt.legend();
    #plt.show()

In [ ]:
import warnings
warnings.filterwarnings("ignore");

for diagnosis in diagnoses.keys():
    graphOverTime(diagnosis);

# Number of people in hospitals by genders and statistical regions

We used another dataset where we get the hospital occupancy based on genders and the statistical regions. We have 13 different statistical regions and 4 different genders that are tracked from year 2003 till 2019.

In [ ]:
from pyaxis import pyaxis
import matplotlib.pyplot as plt
import numpy as np

URL = "https://podatki.nijz.si/DB/NIJZ%20podatkovni%20portal/4%20Zdravstveno%20varstvo/06%20Bolni%C5%A1ni%C4%8Dne%20obravnave/1%20Hospitalizacije%20zaradi%20bolezni/BO01.px"
px = pyaxis.parse(URL, encoding='ANSI')
data = px['DATA']

print(data)

In [ ]:
spol = data['Spol'].values
dataset = data['DATA'].values
regija = data['Statistièna regija'].values
leto = data['Leto'].values

In [ ]:
z=0
spollist= []
for i in spol:
   if i  in spollist:
      z+=1
   else:
        spollist.append(i)

In [ ]:
porazdelitev1 = [0] * len(spollist)
diction = {}
i=0
for spol1 in spollist:
    if spol1 != ('Spol - SKUPAJ'):
        for j in range(0,len(spol)):
            if spol1 == spol[j]:
                porazdelitev1[i] += int(dataset[j])
        diction[spol1] = porazdelitev1[i]
        i+=1

We compared the occupancy based on genders and we can see that in the hospital in the past 15 years, more percentage of the occupancy goes to women also undetermined and unknown are not significally important


In [ ]:
plt.bar(list(diction.keys()), list(diction.values()))
plt.show()

In [ ]:
letolist =[]
z=0
for i in leto:
    if i  in letolist:
      z+=1
    else:
        letolist.append(i)
letolist.sort()

In [ ]:
zenski=dict()
moski=dict()

for letoq in letolist:
    count=0
    count1=0
    for j in range(0,len(leto)-1):
        if letoq == leto[j] and spol[j] == 'Ženski' and regija[j] == "SLOVENIJA":
            count += int(dataset[j])
        if letoq == leto[j] and spol[j] == 'Moški'  and regija[j] == "SLOVENIJA":
            count1 += int(dataset[j])    
    letoq=int(letoq)
    zenski[letoq] = count
    moski[letoq] = count1
zenski = dict(sorted(zenski.items()))
moski = dict(sorted(moski.items()))
print(zenski)
print(moski)

In this part we created a graph of the previous 15 years to compare the occupancy between the men and women and we can see that almost every year the occupancy by the women is higher than the occupancy of the men, also there are years that are quite close almost equal


In [ ]:
plt.plot(list(zenski.keys()), list(zenski.values()),label = "Zenski")
plt.plot(list(moski.keys()), list(moski.values()),label = "Moski")
plt.xlabel('Year')
plt.ylabel('Number of women/men in hospitals')
plt.legend()
plt.show()

In [ ]:
dictot = dict()
for letoq in letolist:
    counttot=0
    for j in range(0,len(leto)):
        if letoq == leto[j] and spol[j] == 'Spol - SKUPAJ' and regija[j]== 'SLOVENIJA':
            counttot += int(dataset[j])
    letoq=int(letoq)
    dictot[letoq] = counttot
dictot = dict(sorted(dictot.items()))

We also plotted a graph of the hospital occupancy over the past 15 years and we can see that the peak of the hospital occupancy is in 2015 and we can see that the least number of used beds are in 2003

In [ ]:
plt.plot(list(dictot.keys()), list(dictot.values()),label = "Slovenija skupaj")
plt.ylabel('Number of people in hospital')
plt.legend()
plt.show()

In [ ]:
regijalist = []
for i in regija:
    regijalist.append(i)
regijalist = list(set(regijalist))
print(regijalist)

In [ ]:
countreg=0
dictreg = dict()

for regija1 in regijalist:
    if regija1 != ('SLOVENIJA'):
        for j in range(0,len(regija)):
            if regija1 == regija[j]:
                countreg += int(dataset[j])
        dictreg[regija1] = countreg
        countreg=0

# statistical region populations
region_population = dict()
region_population["Pomurska"] = 114287
region_population["Podravska"] = 324104
region_population["Koroška"] = 70588
region_population["Savinjska"] = 261851
region_population["Zasavska"] = 41606
region_population["Posavska"] = 70067
region_population["Jugovzhodna Slovenija"] = 144032
region_population["Primorsko-notranjska"] = 52544
region_population["Osrednjeslovenska"] = 564527
region_population["Gorenjska"] = 204670
region_population["Goriška"] = 117616
region_population["Obalno-kraška"] = 115016
region_population["Tujina"] = 160000

relative_dict = dict()
for region in dictreg.keys():
    relative_dict[region] = dictreg[region] / region_population[region]

print(dictreg)
print(relative_dict)

In this part we made a comparison of the occupancy between the statistical regions and we can see that the region "Osrednjoslovenska" is the most occupied, after it come "Podravska" and "Savinjska". The least occupied is the statistical region "Tujina" which says that most of the slovenian people are treated in slovenian hospitals

In [ ]:
index = np.arange(len(list(dictreg.keys())))
plt.bar(list(dictreg.keys()), list(dictreg.values()))
plt.xticks(index, list(dictreg.keys()) , fontsize=10, rotation=75)
plt.show()

In [ ]:
countgrm=0
countgrz=0
dictgr = dict()
for regija1 in regijalist:
    if regija1 != ('SLOVENIJA'):
        for j in range(0,len(regija)):
            if regija1 == regija[j] and spol[j] == "Ženski":
                countgrz += int(dataset[j])
            if regija1 == regija[j] and spol[j] == "Moški":
                countgrm += int(dataset[j])      
        dictgr[regija1] = (countgrz, countgrm)
        countgrz=0
        countgrm=0
print(dictgr)
print(len(dictgr))

Also we compared the males and females based on the statistical regions and we can see that in almost every region there are more women in the hospital than the men 

In [ ]:
m = []
z = []

for val in dictgr.values():
    m.append(val[1])
    z.append(val[0])
X = np.arange(len(m))
plt.bar( X+ 0.00, m, color = 'b', width = 0.25, label="Moski")
plt.bar(X + 0.25, z, color = 'g', width = 0.25, label = "Zenski")
plt.xticks(X, list(dictgr.keys()) , fontsize=10, rotation=75)
plt.legend()
plt.show()

In [ ]:
slovenia_map = json.load(open("map.json", 'r', encoding='utf-8'))

id_dict = dict()
for feature in slovenia_map["features"]:
    id_dict[feature["properties"]["name_1"]] = feature["id"]

map_data = pd.read_csv("data.csv")
map_data["id"] = map_data["Region"].apply(lambda x:id_dict[x])

In [ ]:
fig = px.choropleth(map_data, locations="id", geojson=slovenia_map, color="Value", scope="europe",
                   hover_name="Region")
fig.update_geos(fitbounds="locations", visible=False)
fig.show()